<div style="background-color:#1e1e1e; color:#f2f2f2; padding:20px; font-family:Arial, sans-serif">
    <h2 style="color:#00c8ff;">Step 1: Load & Inspect Dataset</h2>
    <p>In this step we aim to:</p>
    <ul>
        <li>Load the dataset from the Kaggle environment.</li>
        <li>Preview the first few rows to understand its structure.</li>
        <li>Generate a statistical summary of all columns.</li>
        <li>Save these insights as a <b>Dark Theme HTML report</b> for easy viewing.</li>
    </ul>
    <p>This inspection is essential to understand the dataset before any preprocessing or model building.</p>
</div>


In [1]:
import pandas as pd

# Path to dataset in Kaggle environment
path = "/kaggle/input/job-descriptions-2025-tech-and-non-tech-roles/job_dataset.csv"

# Load dataset
df = pd.read_csv(path)

# Preview first 5 rows
print("Preview of dataset:")
print(df.head())

# General info
print("\nDataset Info:")
print(df.info())

# Summary statistics
print("\nSummary Statistics:")
print(df.describe(include='all'))


Preview of dataset:
       JobID           Title ExperienceLevel YearsOfExperience  \
0  NET-F-001  .NET Developer         Fresher               0-1   
1  NET-F-002  .NET Developer         Fresher               0-1   
2  NET-F-003  .NET Developer         Fresher               0-1   
3  NET-F-004  .NET Developer         Fresher               0-1   
4  NET-F-005  .NET Developer         Fresher               0-1   

                                              Skills  \
0  C#; VB.NET basics; .NET Framework; .NET Core f...   
1  C#; .NET Framework basics; ASP.NET; Razor; HTM...   
2  C#; VB.NET basics; .NET Core; ASP.NET MVC; HTM...   
3  C#; .NET Framework; ASP.NET basics; SQL Server...   
4  C#; ASP.NET; MVC; Entity Framework basics; SQL...   

                                    Responsibilities  \
0  Assist in coding and debugging applications; L...   
1  Write simple C# programs under guidance; Suppo...   
2  Contribute to development of small modules; As...   
3  Support in software